In [4]:
%%sh
gcloud container clusters list

NAME        LOCATION  MASTER_VERSION      MASTER_IP      MACHINE_TYPE  NODE_VERSION        NUM_NODES  STATUS
my-cluster  us-east1  1.29.4-gke.1043004  35.227.21.72   e2-small      1.29.4-gke.1043004             RUNNING
virt        us-east1  1.29.4-gke.1043004  34.148.28.218  e2-small      1.29.4-gke.1043004             RUNNING


In [5]:
%%sh
echo 'gcloud log in to kubernetes cluster called virt at location us-east1' | aichat

To log in to a Kubernetes cluster named virt in the us-east1 region using Google Cloud SDK, you can follow these steps:

1. Open your terminal or command prompt.

2. Run the following command to set the context for the cluster:
```
gcloud container clusters get-credentials virt --zone=us-east1
```
This command retrieves cluster credentials and merges them into your Kubernetes configuration file.

3. You should now be logged in to the Kubernetes cluster named virt in the us-east1 region. You can verify this by running the following command:
```
kubectl get nodes
```
This command will list all the nodes in the cluster.

You are now logged in to the Kubernetes cluster named virt in the us-east1 region using Google Cloud SDK.


In [6]:
%%sh
gcloud container clusters get-credentials virt --zone=us-east1

Fetching cluster endpoint and auth data.
kubeconfig entry generated for virt.


In [15]:
%%sh
kubectl cluster-info

Kubernetes control plane is running at https://34.148.28.218
GLBCDefaultBackend is running at https://34.148.28.218/api/v1/namespaces/kube-system/services/default-http-backend:http/proxy
KubeDNS is running at https://34.148.28.218/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy
Metrics-server is running at https://34.148.28.218/api/v1/namespaces/kube-system/services/https:metrics-server:/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


## Deploy KubeVirt

https://kubevirt.io/quickstart_cloud/

In [19]:
%%sh
export VERSION=$(curl -s https://storage.googleapis.com/kubevirt-prow/release/kubevirt/kubevirt/stable.txt)
echo $VERSION
kubectl create -f https://github.com/kubevirt/kubevirt/releases/download/${VERSION}/kubevirt-operator.yaml

v1.2.2
namespace/kubevirt created
customresourcedefinition.apiextensions.k8s.io/kubevirts.kubevirt.io created
priorityclass.scheduling.k8s.io/kubevirt-cluster-critical created
clusterrole.rbac.authorization.k8s.io/kubevirt.io:operator created
serviceaccount/kubevirt-operator created
role.rbac.authorization.k8s.io/kubevirt-operator created
rolebinding.rbac.authorization.k8s.io/kubevirt-operator-rolebinding created
clusterrole.rbac.authorization.k8s.io/kubevirt-operator created
clusterrolebinding.rbac.authorization.k8s.io/kubevirt-operator created


deployment.apps/virt-operator created


In [21]:
%%sh
export VERSION=$(curl -s https://storage.googleapis.com/kubevirt-prow/release/kubevirt/kubevirt/stable.txt)
echo $VERSION
kubectl create -f https://github.com/kubevirt/kubevirt/releases/download/${VERSION}/kubevirt-cr.yaml

v1.2.2
kubevirt.kubevirt.io/kubevirt created


## Verify Components

In [23]:
%%sh
kubectl get kubevirt.kubevirt.io/kubevirt -n kubevirt -o=jsonpath="{.status.phase}"

In [24]:
%%sh
kubectl get all -n kubevirt

NAME                                 READY   STATUS    RESTARTS   AGE
pod/virt-operator-5c58df76cc-glctb   0/1     Pending   0          106s
pod/virt-operator-5c58df76cc-j4xx8   0/1     Pending   0          106s

NAME                            READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/virt-operator   0/2     2            0           107s

NAME                                       DESIRED   CURRENT   READY   AGE
replicaset.apps/virt-operator-5c58df76cc   2         2         0       107s

NAME                            AGE   PHASE
kubevirt.kubevirt.io/kubevirt   38s   


## Install tooling (`virtctl`)

In [25]:
%%sh
nix profile install nixpkgs#kubevirt

this path will be fetched (8.89 MiB download, 48.33 MiB unpacked):
  /nix/store/jsk47l95pxskxpayzbi74l259zzai6bl-kubevirt-1.2.2
copying path '/nix/store/jsk47l95pxskxpayzbi74l259zzai6bl-kubevirt-1.2.2' from 'https://cache.nixos.org'...


In [30]:
%%sh
virtctl 

Available Commands:
  addvolume         add a volume to a running VM
  adm               Administrate KubeVirt configuration.
  completion        Generate the autocompletion script for the specified shell
  console           Connect to a console of a virtual machine instance.
  create            Create a manifest for the specified Kind.
  credentials       Manipulate credentials on a virtual machine.
  expand            Return the VirtualMachine object with expanded instancetype and preference.
  expose            Expose a virtual machine instance, virtual machine, or virtual machine instance replica set as a new service.
  fslist            Return full list of filesystems available on the guest machine.
  guestfs           Start a shell into the libguestfs pod
  guestosinfo       Return guest agent info about operating system.
  help              Help about any command
  image-upload      Upload a VM image to a DataVolume/PersistentVolumeClaim.
  memory-dump       Dump the memory of a